In [145]:
import tensorflow.compat.v1 as tf 
import tensorflow.keras as keras
import pandas as pd
import numpy as np
tf.compat.v1.disable_eager_execution()

In [146]:
def normalize(dataset):
    dataset = dataset.apply(lambda x: (x - x.min(axis = 0)) / (x.max(axis = 0) - x.min(axis = 0)))
    dataset = dataset.loc[:,dataset.any()]
    return dataset

In [168]:
dataset = pd.read_csv("2018.csv" , index_col = 0)
dataset = normalize(dataset.iloc[:,:])
# Splitting the dataset into train 70% and test 30%
features = (dataset.shape[1] - 1)
examples = dataset.shape[0]
train_size = int (0.7 * len(dataset))
(x_train , y_train) = (np.array(dataset.iloc[:train_size,:features]), np.array(dataset.iloc[:train_size,features:]))
(x_test , y_test) = (np.array(dataset.iloc[train_size:,:features]), np.array(dataset.iloc[train_size:,features:]))

,ARR_DELAY
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
49529,1.0
49530,0.0
49531,0.0
49532,1.0


In [149]:
# hyper parameters 
learning_rate = 0.0001
epochs = 50
batch_size = 100
batches = int (train_size / batch_size)

In [154]:
x = tf.placeholder(tf.float32 , [None , features])
y = tf.placeholder(tf.float32 , [None , 1])

W = tf.Variable(tf.zeros([features, 1]))
b = tf.Variable(tf.zeros([1]))

pred = tf.nn.sigmoid(tf.matmul(x,W) + b)  

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = pred))

update = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Define accuracy - Percentage of predictions did we get right
correct_prediction = tf.equal(tf.argmax(pred, 1),tf.argmax(y, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

session = tf.Session()
session.run(tf.global_variables_initializer())

(x_test , y_test) = (np.array(dataset.iloc[train_size:,:features]), np.array(dataset.iloc[train_size:,features:]))


for epoch in range(500):     
    for row in range(batches):        
        batch_x = x_train [row * batch_size : (row + 1) * batch_size]
        batch_y = y_train [row * batch_size : (row + 1) * batch_size]
        session.run(update ,feed_dict = {x : batch_x , y : batch_y})
        _, loss_val = session.run([update, loss],feed_dict={x: batch_x, y: batch_y})
        
    print("Iteration: ",epoch, " Accuracy:", session.run(accuracy, feed_dict={x: x_test, y: y_test}),"loss:",loss_val)

Iteration:  0  Accuracy: 1.0 loss: 0.79813
Iteration:  1  Accuracy: 1.0 loss: 0.79717565
Iteration:  2  Accuracy: 1.0 loss: 0.76197696
Iteration:  3  Accuracy: 1.0 loss: 0.8196602
Iteration:  4  Accuracy: 1.0 loss: 0.7752081
Iteration:  5  Accuracy: 1.0 loss: 0.7264916
Iteration:  6  Accuracy: 1.0 loss: 0.78338933
Iteration:  7  Accuracy: 1.0 loss: 0.7682505
Iteration:  8  Accuracy: 1.0 loss: 0.8239026
Iteration:  9  Accuracy: 1.0 loss: 0.8321539
Iteration:  10  Accuracy: 1.0 loss: 0.7522864
Iteration:  11  Accuracy: 1.0 loss: 0.77940905
Iteration:  12  Accuracy: 1.0 loss: 0.74214953
Iteration:  13  Accuracy: 1.0 loss: 0.7963604
Iteration:  14  Accuracy: 1.0 loss: 0.763468
Iteration:  15  Accuracy: 1.0 loss: 0.7678264
Iteration:  16  Accuracy: 1.0 loss: 0.75756484
Iteration:  17  Accuracy: 1.0 loss: 0.7606062
Iteration:  18  Accuracy: 1.0 loss: 0.74288255
Iteration:  19  Accuracy: 1.0 loss: 0.78145844
Iteration:  20  Accuracy: 1.0 loss: 0.7726476
Iteration:  21  Accuracy: 1.0 loss: 0.7

Iteration:  175  Accuracy: 1.0 loss: 0.71378416
Iteration:  176  Accuracy: 1.0 loss: 0.7174969
Iteration:  177  Accuracy: 1.0 loss: 0.7182917
Iteration:  178  Accuracy: 1.0 loss: 0.7352327
Iteration:  179  Accuracy: 1.0 loss: 0.7223719
Iteration:  180  Accuracy: 1.0 loss: 0.73090774
Iteration:  181  Accuracy: 1.0 loss: 0.72331053
Iteration:  182  Accuracy: 1.0 loss: 0.71120554
Iteration:  183  Accuracy: 1.0 loss: 0.71159285
Iteration:  184  Accuracy: 1.0 loss: 0.72258955
Iteration:  185  Accuracy: 1.0 loss: 0.7268301
Iteration:  186  Accuracy: 1.0 loss: 0.7372057
Iteration:  187  Accuracy: 1.0 loss: 0.71687806
Iteration:  188  Accuracy: 1.0 loss: 0.70677567
Iteration:  189  Accuracy: 1.0 loss: 0.72669816
Iteration:  190  Accuracy: 1.0 loss: 0.7091745
Iteration:  191  Accuracy: 1.0 loss: 0.72755766
Iteration:  192  Accuracy: 1.0 loss: 0.7047157
Iteration:  193  Accuracy: 1.0 loss: 0.7122081
Iteration:  194  Accuracy: 1.0 loss: 0.72565514
Iteration:  195  Accuracy: 1.0 loss: 0.71476656
I

KeyboardInterrupt: 

In [153]:
def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

x = dataset.iloc[7:8,:-1]
logistic_fun(np.matmul(np.array(x),session.run(W)) + session.run(b))[0][0]

0.3439892858528191